In [1]:
from brain_hierarchy import AllenBrainHierarchy
from readCSV_helpers import * #collect_and_analyze_cell_counts, save_results
from pls_helpers import PLS

And we'll need other python functions to easily read and manipulate data and make nice plots:

In [2]:
import pandas as pd
import numpy as np
import os

# import matplotlib.pyplot as plt
import plotly.graph_objects as go

## Load data

Below, you have to specify:
- ```animals_root```: Absolute path to the folder that contains the animal folders.
- ```group_1_name```: A meaningful string for Group 1.
- ```group_2_name```: A meaningful string for Group 2.
- ```group_1_names```: A list of names of the folders corresponding to animals in **Group 1** (e.g., Control group). Indeed, it is necessary to store the results in individual folders for each animal.
- ```group_2_names```: A list of names of the folders corresponding to animals in **Group 2** (e.g., Stress group).

In [3]:
# ####################################### SET PARAMETERS ####################################


animals_root = './data/QuPath_output/'
group_1_name = 'Control'
group_2_name = 'Stress'

data_input_path = './data/python_norm_output/'
data_output_path = './data/python_PLS_output/'
plots_output_path = './plots/python_output/'


# ###########################################################################################


if not(os.path.exists(data_output_path)):
    os.makedirs(data_output_path, exist_ok=True)

if not(os.path.exists(plots_output_path)):
    os.makedirs(plots_output_path, exist_ok=True)

In [4]:
group_1_results = pd.read_csv(os.path.join(data_input_path, f'results_cell_counts_{group_1_name}.csv'), sep='\t', header=[0, 1], index_col=[0,1])
group_2_results = pd.read_csv(os.path.join(data_input_path, f'results_cell_counts_{group_2_name}.csv'), sep='\t', header=[0, 1], index_col=[0,1])

group_1_names = {index[1] for index in group_1_results.index}
group_2_names = {index[1] for index in group_2_results.index}

group_1_tracers = list({cols[0] for cols in group_1_results.columns})
group_2_tracers = list({cols[0] for cols in group_2_results.columns})
assert len(group_1_tracers) == len(group_2_tracers) == 1, "The CSVs in input should have data for one tracer only."
assert group_1_tracers[0] == group_2_tracers[0], f"The data in the CSV of group 1 and 2 should refer to the same tracer. \
Instead group 1 CSV presents data for the tracer '{group_1_tracers[0]}', while group 2 CSV for '{group_2_tracers[0]}'."
tracer = group_1_tracers[0]

The data are stored in ```group_1_results``` and ```group_2_results```:

In [5]:
group_1_results

CFos                           
                         Density Percentage RelativeDensity
root     Control_17C  327.070292   1.000000        1.000000
         Control_18C   33.310720   1.000000        1.000000
         Control_19C  148.701106   1.000000        1.000000
grey     Control_17C  346.050729   0.942186        1.058032
         Control_18C   35.144334   0.941337        1.055046
...                          ...        ...             ...
VISrll6a Control_18C         NaN        NaN             NaN
         Control_19C         NaN        NaN             NaN
VISrll6b Control_17C         NaN        NaN             NaN
         Control_18C         NaN        NaN             NaN
         Control_19C         NaN        NaN             NaN

[3981 rows x 3 columns]

In [8]:
# from https://help.brain-map.org/display/api/Downloading+an+Ontology%27s+Structure+Graph
# StructureGraph id=1
path_to_allen_json = "./data/AllenMouseBrainOntology.json"

branches_to_exclude = ['retina','VS','grv','fiber tracts']
AllenBrain = AllenBrainHierarchy(path_to_allen_json, branches_to_exclude)

# Now, get the selected regions as a variable:
level = 6

selected_regions = AllenBrain.select_at_structural_level(level)
print(f'You selected {len(selected_regions)} regions to analyze.', selected_regions)

selected_regions = AllenBrain.select_at_depth(level)
print(f'You selected {len(selected_regions)} regions to analyze.')

You selected 29 regions to analyze. ['grv of CTX', 'grv of CBX', 'IPF', 'vrt', 'drt', 'cbp', 'lfbst', 'epsc', 'mfbc', 'mfsbshy', 'RAmb', 'CENT', 'CUL', 'AN', 'VENT', 'GENd', 'LAT', 'ATN', 'MED', 'MTN', 'ILM', 'GENv', 'EPI', 'ASO', 'MBO', 'CN', 'DCN', 'PHY', 'VNC']
You selected 534 regions to analyze.


# Partial Least Squares  

The analysis done below is taken from the tutorial written by [Krishnan et al.](https://www.sciencedirect.com/science/article/pii/S1053811910010074).  
Run the 2 cells below to get started.

In [19]:
# PLS
normalization = 'Percentage' # 'Density', 'Percentage' or 'RelativeDensity'
rank = 1

# Create a PLS object
# TODO: see what happens if analyze() does not include the NaN rows
cfosPLS = PLS(group_1_results, group_2_results, group_1_names, group_2_names, selected_regions, tracer, normalization)

# Show the matrix X
cfosPLS.X

,CLA,LA,PA,MEV,SCO,CUN,VTN,PPN,EPd,EPv,...,LGv,MH,LH,SUM,FF,KF,POR,SOCm,SOCl,VCO
Control_17C,0.001643,0.001012,0.001612,0.000009,0.000007,0.001335,0.000019,0.000313,0.004098,0.001517,...,0.000399,0.000406,0.000289,0.000306,0.000155,0.000158,0.000151,0.000394,0.000809,0.002527
Control_18C,0.001394,0.002682,0.001236,0.000053,0.000053,0.001157,0.000026,0.003129,0.003234,0.001420,...,0.004523,0.000973,0.001420,0.001709,0.002051,0.000026,0.002629,0.000500,0.000316,0.000394
Control_19C,0.000207,0.000481,0.000157,0.000017,0.000039,0.000805,0.000000,0.002416,0.002544,0.001102,...,0.000554,0.000436,0.000660,0.000168,0.000235,0.000414,0.003439,0.000705,0.000895,0.003338
Resilient_11R,0.003175,0.001077,0.000289,0.000013,0.000045,0.000462,0.000011,0.000980,0.005760,0.001072,...,0.000783,0.000488,0.001142,0.000205,0.000457,0.000242,0.002206,0.000546,0.000822,0.000580
Resilient_1R,0.001503,0.001057,0.001109,0.000049,0.000035,0.001095,0.000067,0.000713,0.003669,0.001310,...,0.001338,0.001067,0.001306,0.001032,0.000334,0.000713,0.000846,0.000386,0.000758,0.002651
Resilient_2R,0.000952,0.002934,0.004247,0.000045,0.000081,0.001291,0.000005,0.001593,0.003290,0.001453,...,0.003732,0.000302,0.000736,0.001056,0.000519,0.000293,0.000460,0.000203,0.000366,0.000681
Resilient_3R,0.001135,0.001761,0.001815,0.000000,0.000116,0.000358,0.000018,0.002378,0.003916,0.001663,...,0.002718,0.000340,0.000983,0.000286,0.000411,0.000107,0.001833,0.000116,0.000250,0.000027
Resilient_4R,0.001465,0.002024,0.003842,0.000007,0.000056,0.000895,0.000032,0.000827,0.003837,0.002121,...,0.001776,0.000418,0.000477,0.001849,0.000776,0.000681,0.000358,0.000175,0.000355,0.000796
Stress_10S,0.001402,0.002188,0.001618,0.000022,0.000002,0.000755,0.000036,0.001953,0.003609,0.001356,...,0.001625,0.000530,0.000393,0.000539,0.000810,0.000074,0.000098,0.000079,0.000379,0.000146
Stress_13S,0.002250,0.000105,0.000006,0.000012,0.000031,0.000899,0.000155,0.001327,0.005356,0.001277,...,0.000037,0.000124,0.000068,0.000608,0.000167,0.000471,0.001395,0.000831,0.001320,0.001916


In [20]:
# Show the matrix Y
pd.get_dummies(cfosPLS.y).rename(columns={0: group_2_name, 1: group_1_name})

,Stress,Control
Control_17C,0,1
Control_18C,0,1
Control_19C,0,1
Resilient_11R,1,0
Resilient_1R,1,0
Resilient_2R,1,0
Resilient_3R,1,0
Resilient_4R,1,0
Stress_10S,1,0
Stress_13S,1,0


The two matrices printed above (X and Y) illustrate the data on which the PLS is done.  
- ```X:``` The rows in this matrix are the mice. The columns in the matrix are the regions selected for analysis. The values in the matrix are the **normalized value of marked cells: in that region relative to the whole brain.** 
The normalization methods are either:
  + Density
  + Percentage (on the total number of detected marked cells outside of excluded regions)
  + RelativeDensity
- ```Y:``` The rows in this matrix are the mice. The columns in the matrix are the 2 groups. **A value in this matrix is 1 if the mice belongs to the specified group**.

In brief, PLS analyzes the relationship (correlation) between the columns of ```X``` and ```Y```. In our specific case, there will be 2 important outputs:
- **Salience scores**: Each brain region has a salience score. A high salience scores means that the brain region explains much of the correlation between ```X``` and ```Y```.  
- **Singular values**: These are the eigenvalues of the correlation matrix $R = Y^TX$.

## Random permutations to see whether we can differentiate signal from noise. 
Here, we randomly shuffle the group to which a mouse belongs, and calculate the singular values of the permuted dataset.  
From [Krishnan et al.](https://www.sciencedirect.com/science/article/pii/S1053811910010074):  
> The set of all the (permuted) singular values provides a sampling distribution of the singular values under the null hypothesis and, therefore can be used as a null hypothesis test.

*Note: running the cell below will take a few minutes.*

In [21]:
num_permutations = 5000
print(f'Randomly permuting singular values %d times ...'%num_permutations)
s,singular_values = cfosPLS.randomly_permute_singular_values(num_permutations)
print('Done!\n')

Randomly permuting singular values 5000 times ...
Done!



In [22]:
# TODO: move to Plotly

# Plot distribution of singular values
# plt.figure(figsize=(10,4))
# plt.hist(singular_values[:,0],bins=10)
# plt.axvline(cfosPLS.s[0], color='r')
# plt.xlabel('First singular value')
# plt.ylabel('Frequency')
# plt.legend([f'Experiment','Sampling distribution\nunder H0 (%d permutations)'%num_permutations])
# plt.show()

In [23]:
# Calculate p-value = Probability(experiment | H0)
p = (singular_values[:,0] > s[0]).sum() / num_permutations
print('p-value = '+str(p))

p-value = 0.698


## Bootstrap to identify stable salience scores

Here, we use [bootstrapping](https://en.wikipedia.org/wiki/Bootstrapping_(statistics)) (= sampling of the mice in the dataset, with replacement) to get an estimate of which salience scores are stable.

From [Krishnan et al.](https://www.sciencedirect.com/science/article/pii/S1053811910010074):  
> When a vector of saliences is considered generalizable and is kept for further analysis, we need to identify its elements that are stable through resampling. In practice, the stability of an element is evaluated by dividing it by its standard error. [...] To estimate the standard errors, we create bootstrap samples which are obtained by sampling with replacement the observations in and (Efron and Tibshirani, 1986). A salience standard error is then estimated as the standard error of the saliences from a large number of these bootstrap samples (say 1000 or 10000). **The ratios are akin to a Z-score, therefore when they are larger than 2 the corresponding saliences are considered significantly stable.**

*Note: Running the cell below will take a few minutes.*

In [24]:
num_bootstrap = 5000
print(f'Bootstrapping salience scores {num_bootstrap} times...')
u_salience_scores,v_salience_scores = cfosPLS.bootstrap_salience_scores(rank, num_bootstrap)
print('Done!')

Bootstrapping salience scores 5000 times...
Done!


In [25]:
# Plot PLS salience scores
plot_threshold = 1.2 # Only brain regions with a salience higher than plot_threshold are shown. 2 is the significance threshold.
file_title = f'PLS_{tracer}_{normalization}.png'
tp, salient_regions = cfosPLS.plot_salience_scores(plot_threshold, plots_output_path, file_title, AllenBrain.brain_region_dict,
                              fig_width=1000, fig_height=2000)

In [26]:
salient_regions = salient_regions.reset_index()
salient_regions.columns = ['region', 'salience']
salient_regions['salience'] = salient_regions['salience'].abs()
salient_regions = salient_regions.sort_values(by='salience')
salient_regions.to_csv(os.path.join(data_output_path, 'salient_regions.csv'), sep=';', index=False)
salient_regions

,region,salience
44,APr,1.209714
21,GU6b,1.240041
12,SCzo,1.253351
22,VISC1,1.259455
66,VCO,1.270239
...,...,...
48,NDB,2.834433
58,PT,2.834701
20,SSs,3.332497
57,PVT,3.411597


In [27]:
pls_filename = f'PLS_{tracer}_{normalization}_salience_scores.csv'
save_results(v_salience_scores.rename(columns={0:'salience score'}), data_output_path, pls_filename)


! A results_python folder already existed in root. I am overwriting previous results!

Results are saved in ./data/python_PLS_output/

Done!


True

# Plot percentages

In [28]:
# In this case we wanted to normalize it based on the density, rather then the Percentage 
# I didn't modify the various labels in the plot as I was just focused on adapting the code to our dataset, rather then polishing it

threshold = 1e-2 # Only plot bars with value larger than threshold (1e-6, 1e-2, 3)
y_axis_label = 'region_names' # change this to 'acronym' to have acronyms on the y-axis

# Calculate mean values
group_1_df = pd.DataFrame(group_1_results[(tracer,normalization)].rename('cell counts'))
group_1_avg = group_1_df.reset_index().groupby('level_0').mean(numeric_only=True)
group_1_sem = group_1_df.reset_index().groupby('level_0').sem(numeric_only=True)

group_2_df = pd.DataFrame(group_2_results[(tracer,normalization)].rename('cell counts'))
group_2_avg = group_2_df.reset_index().groupby('level_0').mean(numeric_only=True)
group_2_sem = group_2_df.reset_index().groupby('level_0').sem(numeric_only=True)

# Determine which regions to plot  
mean_sum = group_1_avg + group_2_avg
#regs_to_plot = mean_sum[(mean_sum['cell counts']>threshold) & (mean_sum['cell counts'].notnull())].sort_values(by='cell counts').index.to_list()
regs_to_plot = cfosPLS.X.columns.to_list()

# y-axis, with seperate values for each region
y_axis_il, ticklabels = pd.factorize(group_1_df.loc[regs_to_plot].reset_index()['level_0'])
y_axis_bla, ticklabels = pd.factorize(group_2_df.loc[regs_to_plot].reset_index()['level_0'])
if(y_axis_label=='region_names'):
    ticklabels = [AllenBrain.brain_region_dict[reg] for reg in ticklabels]
     
fig = go.Figure()

# Barplot
fig.add_trace(go.Bar(
                     x = group_1_avg.loc[regs_to_plot]['cell counts'],
                     name = f'{group_1_name} mean',
                     error_x = dict(
                         type='data',
                         array=group_1_sem.loc[regs_to_plot]['cell counts']
                     )
              )
)

fig.add_trace(go.Bar(
                     x = group_2_avg.loc[regs_to_plot]['cell counts'],
                     name = f'{group_2_name} mean',
                     error_x = dict(
                         type='data',
                         array=group_2_sem.loc[regs_to_plot]['cell counts']
                     )
              )
)

fig.update_layout(barmode='group', colorway=['rgb(0,255,0)', 'rgb(255,0,0)'])

# Scatterplot (animals)
fig.add_trace(go.Scatter(
                    mode = 'markers',
                    y = y_axis_il - 0.2,
                    x = group_1_df.loc[regs_to_plot]['cell counts'],
                    name = f'{group_1_name} animals',
                    opacity=0.5,
                    marker=dict(
                        color='rgb(0,255,0)',
                        size=5,
                        line=dict(
                            color='rgb(0,0,0)',
                            width=1
                        )
                    )
              )
)

fig.add_trace(go.Scatter(
                    mode = 'markers',
                    y = y_axis_bla + 0.2,
                    x = group_2_df.loc[regs_to_plot]['cell counts'],
                    name = f'{group_2_name} animals',
                    opacity=0.5,
                    marker=dict(
                        color='rgb(255,0,0)',
                        size=5,
                        line=dict(
                            color='rgb(0,0,0)',
                            width=1
                        )
                    )
              )
)

# Figure title
title = ''
if normalization == 'RelativeDensity':
    title = f'[#{tracer} / area] / [{tracer} (brain) / area (brain)].'
elif normalization == 'Density':
    title = f'[#{tracer} / area]'
elif normalization == 'Percentage':
    title = f'[#{tracer} / brain]'

# Update layout
fig.update_layout(
    title = title,
    yaxis = dict(
        tickmode = 'array',
        tickvals = np.arange(0,len(regs_to_plot)),
        ticktext = ticklabels
    ),
    xaxis=dict(
        title = f'{tracer} density (relative to brain)'
    ),
    width=900, height=5000,
    hovermode="x unified",
    yaxis_range = [-1,len(regs_to_plot)+1]
)

fig.show()

# Save figure as PNG
file_title = 'barplot_' + tracer + '_' + normalization + 'CvS.png'
fig.write_image(os.path.join(plots_output_path, file_title))